# AI Economy Sentiment Agent - Production Analysis

## Overview
This notebook orchestrates the AI Economy Sentiment Agent with clean, modular code.

### Multi-Agent Workflow:
- **Agent 1**: Curates most relevant news articles from 30+ fetched
- **Agent 2**: Selects most relevant Polymarket prediction markets from 100+ fetched
- **Agent 3**: Analyzes sentiment of selected articles
- **Agent 4**: Generates comprehensive analysis integrating all data

---

## Setup

Import all required modules from our production package.

In [1]:
# Add current directory to path
import sys
sys.path.insert(0, '.')

from src.data_collectors import (
    get_market_data, get_sector_performance, get_fear_greed_index, get_polymarket_data, 
    get_earnings_sentiment, is_market_open, get_economic_indicators, get_news_articles
)
from src.ai_agents import (
    select_relevant_articles, select_relevant_polymarkets, analyze_news_sentiment,
    generate_comprehensive_analysis
)

print("Modules imported successfully!")

Modules imported successfully!


---
## Step 1: Collect Market Data

In [2]:
market_is_open = is_market_open()
print(f"Market Status: {'OPEN' if market_is_open else 'CLOSED'}\n")
market_data = get_market_data()

# Display market data with percent changes
print("\n📊 MAJOR INDICES:")
for name, data in market_data.items():
    if data:
        status = "🟢 LIVE" if data.get('is_live') else "⚪ Last Close"
        change_pct = data['daily_change_pct']
        emoji = "🟢" if change_pct > 0 else "🔴" if change_pct < 0 else "⚪"
        print(f"{emoji} {name:15s}: ${data['current_price']:8.2f} ({change_pct:+.2f}%) [{status}]")

Market Status: OPEN

Fetching market data...


📊 MAJOR INDICES:
🟢 S&P 500        : $ 6829.05 (+0.93%) [🟢 LIVE]
🟢 Nasdaq-100     : $  615.65 (+1.11%) [🟢 LIVE]
🔴 VIX            : $   17.37 (-6.41%) [🟢 LIVE]


---
## Step 2: Get Sector Performance (All 11 S&P Sectors)

Sector rotation reveals market positioning and investor sentiment.

In [ ]:
sector_data = get_sector_performance()

# Display sector performance ranked by daily change
print("\n📊 SECTOR PERFORMANCE (Ranked by Daily Change):")
print("="*70)

valid_sectors = [(s, d) for s, d in sector_data.items() if d and isinstance(d, dict) and 'daily_change_pct' in d]
if valid_sectors:
    sorted_sectors = sorted(valid_sectors, key=lambda x: x[1]['daily_change_pct'], reverse=True)
    
    for sector, data in sorted_sectors:
        daily = data['daily_change_pct']
        weekly = data['week_change_pct']
        emoji = "🟢" if daily > 0 else "🔴" if daily < 0 else "⚪"
        print(f"{emoji} {sector:25s} ({data['ticker']}): {daily:+.2f}% (day)  |  {weekly:+.2f}% (week)")

Fetching sector performance...
  Leader (Daily): Technology (+1.49%)
  Laggard (Daily): Healthcare (-0.05%)


📊 SECTOR PERFORMANCE (Ranked by Daily Change):
🟢 Technology                (XLK): +1.49% (day)  |  +4.59% (week)
🟢 Technology                (XLK): +1.49% (day)  |  +4.59% (week)
🟢 Materials                 (XLB): +1.41% (day)  |  +5.43% (week)
🟢 Materials                 (XLB): +1.41% (day)  |  +5.43% (week)
🟢 Energy                    (XLE): +1.22% (day)  |  +0.94% (week)
🟢 Energy                    (XLE): +1.22% (day)  |  +0.94% (week)
🟢 Utilities                 (XLU): +1.20% (day)  |  +2.11% (week)
🟢 Utilities                 (XLU): +1.20% (day)  |  +2.11% (week)
🟢 Consumer Staples          (XLP): +1.16% (day)  |  +2.47% (week)
🟢 Consumer Staples          (XLP): +1.16% (day)  |  +2.47% (week)
🟢 Financials                (XLF): +1.10% (day)  |  +3.93% (week)
🟢 Financials                (XLF): +1.10% (day)  |  +3.93% (week)
🟢 Industrials               (XLI): +0.88% (day)  | 

---
## Step 3: Get Fear & Greed Index


In [4]:
fear_greed_score, fear_greed_rating = get_fear_greed_index()
if fear_greed_score:
    print(f"Fear & Greed Index: {fear_greed_score:.1f} - {fear_greed_rating.upper()}")

Fear & Greed Index: 18.7 - EXTREME FEAR


---
## Step 3: Get Polymarket Predictions

In [5]:
# Fetch raw Polymarket data
polymarket_raw = get_polymarket_data()

# Use AI to select most relevant markets
polymarket_data = select_relevant_polymarkets(polymarket_raw)

if 'error' not in polymarket_data and polymarket_data.get('selected_markets'):
    print(f"\nPolymarket Predictions (AI-Selected Markets):\n")
    print(f"AI selected {len(polymarket_data['selected_markets'])} most relevant prediction markets\n")
    
    for i, market in enumerate(polymarket_data['selected_markets'], 1):
        prob = market.get('probability')
        relevance = market.get('relevance', '')
        
        print(f"{i}. {market['question']}")
        if prob is not None:
            prob_pct = prob * 100
            print(f"   Probability: {prob_pct:.1f}%")
        if relevance:
            print(f"   Relevance: {relevance}")
        print()
else:
    print(f"\n[INFO] Polymarket: {polymarket_data.get('note', 'No predictions found')}")
    print(f"       Continuing analysis without Polymarket data...")
    # Set to empty dict so analysis can continue
    if 'stats' in polymarket_data:
        stats = polymarket_data['stats']
        print(f"       (Found {stats.get('total_economic', 0)} economic markets, but all were closed)")


Fetching Polymarket data from Gamma API...
  Trying tag 'Economics'...
  [OK] Got 500 markets with tag 'Economics'
  Economic markets for AI: 45

AI selecting 15 most relevant markets from 45 candidates...
✓ Selected 15 Polymarket predictions


Polymarket Predictions (AI-Selected Markets):

AI selected 15 most relevant prediction markets

1. Brazil unemployment below 6.3% Q4 2025?
   Probability: 95.7%
   Relevance: Brazil unemployment below 6.3% Q4 2025 prediction job market stability outlook

2. Will 3 Fed rate cuts happen in 2025?
   Probability: 83.0%
   Relevance: Federal rate cutting trend influences economic growth, inflation expectations

3. Will Trump refrain from announcing a next Fed Chair in 2025?
   Probability: 46.0%
   Relevance: No Fed leadership announcement impacts monetary policy uncertainty

4. Will 2 Fed rate cuts happen in 2025?
   Probability: 15.0%
   Relevance: Dual cut scenario affects interest rates, borrowing costs, business investment

5. Will the 10-year T

---
## Step 4: Get Earnings Sentiment


In [6]:
earnings_data = get_earnings_sentiment()

if 'error' not in earnings_data:
    print(f"Earnings Sentiment Analysis:\n")
    print(f"Analyzed {earnings_data.get('companies_analyzed', 0)} major companies\n")
    print("AI Analysis:")
    print(earnings_data.get('ai_sentiment_analysis', 'No analysis available'))
else:
    print(f"\n[INFO] Earnings Data: {earnings_data.get('note', 'Unknown error')}")
    if 'suggestion' in earnings_data:
        print(f"       {earnings_data['suggestion']}")
    print(f"       Continuing analysis with other data sources...")
    # Set earnings_data to empty dict so analysis can continue
    earnings_data = {'note': 'Data unavailable due to rate limiting'}


Fetching earnings data...
  [OK] AAPL: Apple Inc.
  [OK] MSFT: Microsoft Corporation
  [OK] GOOGL: Alphabet Inc.
  [OK] AMZN: Amazon.com, Inc.
  [OK] NVDA: NVIDIA Corporation
  [OK] TSLA: Tesla, Inc.
  [OK] META: Meta Platforms, Inc.
[OK] Earnings sentiment analyzed for 7 companies
Earnings Sentiment Analysis:

Analyzed 7 major companies

AI Analysis:
**Overall Market Sentiment:** Bullish  

**Key Observations**  
- **Dominant Growth in the “Big 5”** – Apple, Microsoft, Alphabet, Amazon, and NVIDIA all report positive revenue and earnings growth, with NVIDIA delivering the strongest upside (≈ +38 % to target).  
- **Mixed Earnings Outlook for TSLA & META** – Both companies show negative earnings growth but maintain high revenue growth and healthy margins; TSLA’s current price already exceeds its target, while META still offers ≈ +32 % upside.  
- **Strong Analyst Consensus** – Six of the seven firms carry “buy” or “strong_buy” ratings, underscoring confidence in continued expansion des

---
## Step 5: Fetch Economic Indicators

In [7]:
economic_data = get_economic_indicators()


Fetching economic indicators from FRED...

[OK] Federal Funds Rate: 3.88
[OK] CPI Index: 324.368
  (Note: For YoY inflation %, use official BLS reports)
[OK] Unemployment Rate: 4.4
[OK] 10-Year Treasury Yield: 4.04
[OK] Non-Farm Payrolls (thousands): 159626.0
  (Most recent monthly employment figure)


---
## Step 5: Collect & Curate News

In [8]:
news_articles = get_news_articles()
selected_articles = select_relevant_articles(news_articles)
print(f"\n{len(selected_articles)} articles selected")

Fetching news articles...

Fetching Economy news...
  [OK] Analysis: China’s Economy and Its Influence on Global Markets - U.S. B...
  [OK] Weekly market wrap - Edward Jones...
  [OK] Market Volatility Underscores Epic Buildup of Global Risk - The New Yo...
  [OK] 2026 Outlooks: Market and Economic Forecasts - Morgan Stanley...
  [OK] What’s Happening in the Markets This Week - Morningstar...
  [OK] Japan policymakers agree to watch market with 'strong sense of urgency...
  [OK] Global Economic Outlook: November 2025 - S&P Global...
  [OK] America’s huge mortgage market is slowly dying - The Economist...
  [OK] The ‘S&P 493’ reveals a very different U.S. economy - The Washington P...
  [OK] NYSE Content Advisory: Pre-Market Update + U.S. Economy Adds 119,000 J...
  [OK] Global risk of shocks is unprecedentedly high, warns ECB - Green Centr...
  [OK] Will we reach escape velocity in 2026? - UBS...
  [OK] S&P 500 Index: Evolution, Significance, and Economic Impact - Investop...
  [OK] Te

---
## Step 6: AI Sentiment Analysis

In [9]:
print("="*60 + "\nNEWS SENTIMENT ANALYSIS\n" + "="*60 + "\n")
news_sentiment = analyze_news_sentiment(selected_articles)
print(news_sentiment)

NEWS SENTIMENT ANALYSIS

Analyzing news sentiment with AI...

**1. Overall Sentiment**  
- **Neutral‑to‑slightly bearish** overall.  
  - Positive: robust job growth (119 k new U.S. jobs in Sept – *Article 3*) and a consensus that a December rate cut is likely (*Articles 5, 6, 9*).  
  - Negative: high inflation persists, weak labor‑market softness continues, and Fed policy is highly divided (*Articles 4, 7, 8, 10*).  
- **Confidence**: ~65 % that the market will stay in a cautious stance for the next 3–6 months.

---

**2. Key Themes Across All Articles**

| Theme | What the articles say | Article support |
|-------|-----------------------|-----------------|
| **Inflation pressures remain high** | PPI and headline CPI still above 2‑year lows; inflation data delays create uncertainty. | 4, 5, 11 |
| **Mixed labor‑market data** | Strong job creation (119 k) but wages are stagnating; the labor market is “struggling” (weak upside). | 3, 4, 12 |
| **Fed’s policy crossroads** | Fed official

---
## Step 7: Comprehensive Market Analysis

In [10]:
print("\n" + "="*60 + "\nCOMPREHENSIVE MARKET ANALYSIS\n" + "="*60 + "\n")
analysis = generate_comprehensive_analysis(
    market_data=market_data,
    sector_data=sector_data,
    fear_greed=(fear_greed_score, fear_greed_rating),
    polymarket=polymarket_data,
    earnings=earnings_data,
    economic_data=economic_data,
    news_sentiment=news_sentiment,
    market_is_open=market_is_open
)
print(analysis)


COMPREHENSIVE MARKET ANALYSIS

**📈 Market Snapshot – 26 Nov 2025**  
| Item | Value |
|------|-------|
| **S&P 500** | **$6,828.35** (+0.92 %) |
| **Nasdaq‑100** | **$615.59** (+1.10 %) |
| **VIX** | **17.45** (‑5.98 %) |
| **Fed Funds Target** | **3.88 %** |
| **10‑Yr Treasury Yield** | **4.04 %** |
| **CPI (Index)** | **324.368** |
| **Unemployment** | **4.40 %** |
| **NFP (Sep 25)** | **+159 k** |

> **Bottom line:** The US equity market is in a mild‑bullish stance with a very low “fear” score (18.7/100 on the Fear‑Greed Index).  The economy still shows strong employment, but inflation remains stubbornly high.  The Fed is clearly in a tight‑but‑not‑over‑tight position, with market consensus pointing toward a rate‑cut cycle in 2025.  

---

## 1. Market Sentiment & Investor Psychology  

| Indicator | Interpretation |
|-----------|----------------|
| **Fear‑Greed Index (18.7 < 30)** | Extreme fear – investors are cautious, risk‑aversion high.  Despite the bullish numbers, the market